# Apparent Magnitude Validation

In [ ]:
import pandas as pd
import numpy as np
import astropy.units as u
from astroquery.jplhorizons import Horizons
from sorcha.modules.PPCalculateApparentMagnitudeInFilter import PPCalculateApparentMagnitudeInFilter
import matplotlib.pyplot as plt

To test the calculation of the apparent magnitude in the code, we can compare them to the apparent magnitudes calculated by JPL Horizons.

First, let's get the JPL Horizons ephemeris for a test object. PPCalculateApparentMagnitudeInFilter uses sbpy's photometry module to calculate phase functions, and sbpy's unit tests use 24 Themis as a test object. We will do the same.

In [ ]:
obj = Horizons(id='Themis', id_type='name', location='I11',

               epochs={'start':'2021-01-01', 'stop':'2023-01-01',

                       'step':'1d'})

eph = obj.ephemerides(quantities='9,19,20,43')
jpl_eph = eph.to_pandas()
jpl_eph

This needs to be turned into a form the function can understand. Values for G1, G2 and G12 are from Muinonen et al. (2010).

In [ ]:
observations_df = pd.DataFrame({'MJD':jpl_eph['datetime_jd'],
                                'H_filter': jpl_eph['H'],
                                'GS': jpl_eph['G'],
                                'G1': np.zeros(len(jpl_eph['G'])) + 0.62,
                                'G2': np.zeros(len(jpl_eph['G'])) + 0.14,
                                'G12': np.zeros(len(jpl_eph['G'])) + 0.68,
                                'JPL_mag': jpl_eph['V'],
                                'AstRange(km)': jpl_eph['r'] * 1.495978707e8,
                                'Ast-Sun(km)': jpl_eph['delta'] * 1.495978707e8,
                                'Sun-Ast-Obs(deg)': jpl_eph['alpha_true']})

In [ ]:
observations_df

Now we calculate the magnitude using the various models in PPCalculateApparentMagnitudeInFilter.

In [ ]:
observations_df = PPCalculateApparentMagnitudeInFilter(observations_df.copy(), 'HG', 'r', 'HG_mag')
observations_df = PPCalculateApparentMagnitudeInFilter(observations_df.copy(), 'HG12', 'r', 'HG12_mag')
observations_df = PPCalculateApparentMagnitudeInFilter(observations_df.copy(), 'HG1G2', 'r', 'HG1G2_mag')

In [ ]:
observations_df

Now we can plot the magnitudes and compare them.

Note that we do not expect any of the calculated magnitudes to match JPL Horizons exactly. JPL Horizons uses the IAU simplification of the HG model to calculate apparent magnitude, while sbpy uses the original HG formulation from Bowell et al. (1989). However, they should all be a close match.

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(observations_df["MJD"] - 2459000, observations_df["JPL_mag"], linestyle="", marker="x", label="JPL")
ax.plot(observations_df["MJD"] - 2459000, observations_df["HG_mag"], label="HG")
ax.plot(observations_df["MJD"] - 2459000, observations_df["HG12_mag"], label="HG12")
ax.plot(observations_df["MJD"] - 2459000, observations_df["HG1G2_mag"], label="HG1G2")
ax.legend()
ax.set_xlabel("MJD - 2459000")
ax.set_ylabel("apparent magnitude")
ax.set_ylim((11.75, 14))

To test the linear phase function model, we simply define a slope. We will use the same values for Themis and arbitrarily choose S to be 0.04.

In [ ]:
H = observations_df['H_filter'].values
alpha = observations_df['Sun-Ast-Obs(deg)'].values
r = observations_df['AstRange(km)'].values / 1.495978707e8
delta = observations_df['Ast-Sun(km)'].values / 1.495978707e8
S = np.zeros(len(H)) + 0.04

observations_df["S"] = S

The expected apparent magnitude will thus take the form:

$m = H + 5 \log_{10}(\Delta) + 5 \log_{10}(r) + S\alpha$

In [ ]:
linear_mag_calc = 5. * np.log10(delta) + 5. * np.log10(r) + H + (S * alpha)

Calculating using the linear phase function model in PPCalculateApparentMagnitudeInFilter...

In [ ]:
observations_df = PPCalculateApparentMagnitudeInFilter(observations_df.copy(), 'linear', 'r', 'linear_mag')

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(observations_df["MJD"] - 2459000, linear_mag_calc, linestyle="", marker="x", label="calculated")
ax.plot(observations_df["MJD"] - 2459000, observations_df["linear_mag"], label="function")
ax.legend()
ax.set_xlabel("MJD - 2459000")
ax.set_ylabel("apparent magnitude")
ax.set_ylim((11.75, 14))